<a href="https://colab.research.google.com/github/ArezaB/Reza/blob/main/pointsforgoogleearth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import pandas as pd
from pyproj import Proj, transform

def utm_to_latlon(easting, northing, zone_number=40, northern_hemisphere=True):
    in_proj = Proj(init=f'epsg:326{zone_number}')
    out_proj = Proj(init='epsg:4326')  # WGS84

    lon, lat = transform(in_proj, out_proj, easting, northing)
    return lat, lon

def convert_utm_to_latlon(input_csv, output_csv):
    # Check if the input file exists
    if os.path.exists(input_csv):
        # Read the CSV file from the local file path
        df = pd.read_csv(input_csv)

        # Display the first few rows of the DataFrame
        print(df.head())

        # Convert UTM to Lat/Lon
        df['Latitude'], df['Longitude'] = zip(*df.apply(lambda row: utm_to_latlon(row['Easting'], row['Northing']), axis=1))

        # Save the results to a new CSV file
        df.to_csv(output_csv, index=False)

        print(f"Conversion completed. Results saved to {output_csv}")
    else:
        print(f"The input file {input_csv} does not exist.")

# Replace with your actual file paths
input_file_path = '/content/input_file.csv'
output_file_path = '/content/output_file.csv'

convert_utm_to_latlon(input_file_path, output_file_path)




      Easting     Northing
0  571196.321  3172959.275
1  571220.512  3172962.611
2  571239.107  3172944.527
3  571304.923  3172996.451
4  571305.584  3173026.217
Conversion completed. Results saved to /content/output_file.csv


In [24]:
!pip install geopandas shapely


In [26]:
!pip install simplekml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 784.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=cdf5de26d6126e099edef61ee6dac53b63127bbac89c205f900de89bc30c01f9
  Stored in directory: /root/.cache/pip/wheels/16/7c/e6/621a95f24bd1ff987368f1c37881ffe013433ff6b413d87fcb
Successfully built simplekml


In [28]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import simplekml

# Load the CSV file with the converted UTM to Lat/Lon coordinates
file_path = '/content/output_file.csv'
df = pd.read_csv(file_path)

# Create a GeoDataFrame with Point geometries
geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')  # WGS84

# Create a LineString connecting the points in order
line = LineString(gdf['geometry'])

# Save the LineString to a KML file using simplekml
output_kml_path = '/content/line.kml'
kml = simplekml.Kml()
linestring = kml.newlinestring(name='Connected Line', coords=list(line.coords))
kml.save(output_kml_path)

print(f"Connected line saved to {output_kml_path}")



Connected line saved to /content/line.kml
